In [1]:
from joblib import Parallel, delayed
from pyaspeller import YandexSpeller
from pymystem3 import Mystem
import os
import pandas as pd
from tqdm import tqdm
speller = YandexSpeller()
mystem = Mystem(grammar_info=False)

In [58]:
TITLES_DIR = './DOCS_TITLES/'

def proc(line):
    t = line[:-1].strip().lower().replace(u'ё', u'е').split('\t')
    doc_id = t[0]
    if len(t) > 1:
        title = t[1]
        fixed = speller.spelled(title)

        spell_res = fixed + '\n'

        lemmas = mystem.lemmatize(fixed)
        stemmed = ''.join(lemmas)

        title_res = stemmed + '\n'
        
        with open(os.path.join(TITLES_DIR, doc_id + '.txt'), 'w') as save:
            save.write(spell_res)
            save.write(title_res)
    else:
        with open(os.path.join(TITLES_DIR, str(doc_id) + '.txt'), 'w') as save:
            save.write('')
            save.write('')

In [3]:
from pymystem3 import Mystem
from tqdm import tqdm

docs_filename = '/mnt/HDD/FINAL_TS/docs.tsv'
spell_titles = './DOCS/Spell_titles.tsv'
spell_stem_titles = './DOCS/Spell_stem_titles.tsv'

docs_num = 582167

with open(docs_filename, 'r') as docs_file:
    Parallel(n_jobs=-1)(delayed(proc)(line) for line in tqdm(docs_file, total=docs_num))

100%|██████████| 582167/582167 [9:03:25<00:00, 17.85it/s]  


### Read

In [4]:
import os
import tqdm
import pandas as pd

In [65]:
df_speller = pd.DataFrame(columns=['Id', 'Title'])
df_speller_stemmer = pd.DataFrame(columns=['Id', 'Title'])

In [66]:
lst_speller = []
lst_speller_stemmer = []

In [67]:
for doc in tqdm.tqdm(os.listdir('./DOCS_TITLES/')):
    doc_id = doc.split('.')[0]
    with open(os.path.join('./DOCS_TITLES', doc), 'r') as f:
        title_speller = f.readline()
        title_speller_stemmer = f.readline()
        
        lst_speller.append([doc_id, title_speller])
        lst_speller_stemmer.append([doc_id, title_speller_stemmer])

100%|██████████| 582167/582167 [01:23<00:00, 6997.07it/s]


In [68]:
df_speller = pd.DataFrame(lst_speller, columns =['Id', 'Title'])

In [69]:
df_speller_stemmer = pd.DataFrame(lst_speller_stemmer, columns =['Id', 'Title'])

In [70]:
df_speller_stemmer.to_csv('titles_speller_stemmer.tsv',sep='\t',index=False,header=False)

In [71]:
df_speller.to_csv('titles_speller.tsv',sep='\t',index=False,header=False)

### Queries

In [60]:
queries = pd.read_csv('./Data/queries.tsv', header=None, sep='\t')

In [74]:
query_speller = []
query_speller_stemmer = []

for row in tqdm.tqdm_notebook(queries.iterrows()):
    queiry_id = row[0]
    queiry = row[1][1].strip().lower().replace(u'ё', u'е')
    
    spelled = speller.spelled(queiry)
    lemmas = mystem.lemmatize(spelled)
    stemmed = ''.join(lemmas)
        
    query_speller.append([queiry_id, spelled])
    query_speller_stemmer.append([queiry_id, stemmed])

/home/andrey/anaconda3/envs/matcher/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


0it [00:00, ?it/s]

In [184]:
query_speller_stemmer[5252]

[5252, 'сколько надо ездить со знак начинающий водитель\n']

In [77]:
df_query_speller = pd.DataFrame(query_speller, columns =['Id', 'Query'])
df_query_speller_stemmer = pd.DataFrame(query_speller_stemmer, columns =['Id', 'Query'])

In [185]:
df_query_speller

,Id,Query
0,0,13 причин почему
1,1,1 положительный и 1 отрицательный могут ли
2,2,2016 действует ли зао рождественская мануфактура
3,3,1 месяц после операции на кишечнике диета что ...
4,4,2 правды 1 ложь что можно придумать
...,...,...
6306,6306,является ли тойота харриер внедорожником
6307,6307,як можно очистить крейду
6308,6308,являются ли реактивы медицинскими изделиями
6309,6309,являются ли словообразовательными парами слова...


In [83]:
df_query_speller.to_csv('query_speller.tsv',sep='\t',index=False,header=False)

In [80]:
df_query_speller_stemmer['Query'] = df_query_speller_stemmer['Query'].apply(lambda x: x.strip())

In [82]:
df_query_speller_stemmer.to_csv('query_speller_stemmer.tsv',sep='\t',index=False,header=False)

### BM25

In [132]:
titles = pd.read_csv('./titles_speller.tsv', sep='\t', header=None)
titles.columns=['Id', 'Title']

In [134]:
titles

,Id,Title
0,334761,ответы mail ru кто родился 4 июля\n
1,56349,наращивание ногтей акрилом описание достоинств...
2,551990,как узнать есть ли что то в собственности zaza...
3,224905,relp криминалистическое исследование речевого ...
4,389288,николай козлов синтон психология тренинги розе...
...,...,...
582162,458053,голод сергей львович москвин голод книга подго...
582163,474067,как сделать фотографию снимок с экрана монитор...
582164,179069,селективная парфюмерия нишевая парфюмерия стат...
582165,539701,10 установленных плагинов wordpress в блоге но...


In [135]:
queries = pd.read_csv('./query_speller.tsv', sep='\t', header=None)
queries.columns = ['Id', 'Query']

In [137]:
queries

,Id,Query
0,0,13 причин почему
1,1,1 положительный и 1 отрицательный могут ли
2,2,2016 действует ли зао рождественская мануфактура
3,3,1 месяц после операции на кишечнике диета что ...
4,4,2 правды 1 ложь что можно придумать
...,...,...
6306,6306,является ли тойота харриер внедорожником
6307,6307,як можно очистить крейду
6308,6308,являются ли реактивы медицинскими изделиями
6309,6309,являются ли словообразовательными парами слова...


In [138]:
queries['Id'] = queries['Id'].apply(lambda x: int(x))

In [139]:
titles['Title']

0                       ответы mail ru кто родился 4 июля\n
1         наращивание ногтей акрилом описание достоинств...
2         как узнать есть ли что то в собственности zaza...
3         relp криминалистическое исследование речевого ...
4         николай козлов синтон психология тренинги розе...
                                ...                        
582162    голод сергей львович москвин голод книга подго...
582163    как сделать фотографию снимок с экрана монитор...
582164    селективная парфюмерия нишевая парфюмерия стат...
582165    10 установленных плагинов wordpress в блоге но...
582166                         сто великих мифов и легенд\n
Name: Title, Length: 582167, dtype: object

In [140]:
titles['Title'] = titles['Title'].astype('str').apply(lambda x: x.strip())
titles['Id'] = titles['Id'].apply(lambda x: int(x))
titles = titles.sort_values(by=['Id'])

In [141]:
from rank_bm25 import BM25Okapi, BM25L, BM25Plus

In [142]:
corpus = list(titles['Title'])
tokenized_corpus = [doc.split(" ") for doc in corpus]

bm25_Okapi = BM25Okapi(tokenized_corpus)
bm25_L = BM25L(tokenized_corpus)
bm25_Plus = BM25Plus(tokenized_corpus)

In [159]:
sample_df = pd.read_csv('./Data/sample.csv', sep=',')
sample_df.columns = ['query_id', 'url_id']

In [160]:
marks_df = pd.read_csv('./Data/train.marks.tsv', sep='\t', header=None)
marks_df.columns = ['query_id', 'url_id', 'mark']
marks_df = marks_df.drop(columns=['mark'])

sample_df = sample_df.append(marks_df)

In [161]:
sample_df['BM25Okapi'] = None
sample_df['BM25L'] = None
sample_df['BM25Plus'] = None

In [162]:
sample_df

,query_id,url_id,BM25Okapi,BM25L,BM25Plus
0,0,340485,None,None,None
1,0,68106,None,None,None
2,0,237314,None,None,None
3,0,203791,None,None,None
4,0,53265,None,None,None
...,...,...,...,...,...
202074,6305,63981,None,None,None
202075,6305,354802,None,None,None
202076,6305,275960,None,None,None
202077,6305,338427,None,None,None


In [163]:
max(sample_df['url_id'])

582166

In [164]:
queries

,Id,Query
0,0,13 причин почему
1,1,1 положительный и 1 отрицательный могут ли
2,2,2016 действует ли зао рождественская мануфактура
3,3,1 месяц после операции на кишечнике диета что ...
4,4,2 правды 1 ложь что можно придумать
...,...,...
6306,6306,является ли тойота харриер внедорожником
6307,6307,як можно очистить крейду
6308,6308,являются ли реактивы медицинскими изделиями
6309,6309,являются ли словообразовательными парами слова...


In [165]:
for query in tqdm.tqdm_notebook(set(sample_df['query_id'])):
    docs_id = list(sample_df[sample_df['query_id'] == query]['url_id'])
    q = queries[queries['Id'] == query]['Query'][query]
    tokenized_query = q.split(" ")

    bm25_1 = bm25_Okapi.get_batch_scores(tokenized_query, docs_id)
    bm25_2 = bm25_L.get_batch_scores(tokenized_query, docs_id)
    bm25_3 = bm25_Plus.get_batch_scores(tokenized_query, docs_id)
    
    sample_df.loc[sample_df['query_id'] == query, 'BM25Okapi'] = bm25_1
    sample_df.loc[sample_df['query_id'] == query, 'BM25L'] = bm25_2
    sample_df.loc[sample_df['query_id'] == query, 'BM25Plus'] = bm25_3

/home/andrey/anaconda3/envs/matcher/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/6311 [00:00<?, ?it/s]

In [166]:
sample_df.to_csv('./BM25/BM25_Spelled.tsv',sep='\t',index=False)

In [167]:
### ТОЖЕ САМОЕ, НО С СТЕММЕРОМ ###

In [168]:
titles = pd.read_csv('./titles_speller_stemmer.tsv', sep='\t', header=None)
titles.columns=['Id', 'Title']

queries = pd.read_csv('./query_speller_stemmer.tsv', sep='\t', header=None)
queries.columns = ['Id', 'Query']
queries['Id'] = queries['Id'].apply(lambda x: int(x))

titles['Title'] = titles['Title'].astype('str').apply(lambda x: x.strip())
titles['Id'] = titles['Id'].apply(lambda x: int(x))
titles = titles.sort_values(by=['Id'])

In [169]:
from rank_bm25 import BM25Okapi, BM25L, BM25Plus

In [170]:
corpus = list(titles['Title'])
tokenized_corpus = [doc.split(" ") for doc in corpus]

bm25_Okapi = BM25Okapi(tokenized_corpus)
bm25_L = BM25L(tokenized_corpus)
bm25_Plus = BM25Plus(tokenized_corpus)

In [171]:
sample_df = pd.read_csv('./Data/sample.csv', sep=',')
sample_df.columns = ['query_id', 'url_id']

marks_df = pd.read_csv('./Data/train.marks.tsv', sep='\t', header=None)
marks_df.columns = ['query_id', 'url_id', 'mark']
marks_df = marks_df.drop(columns=['mark'])

sample_df = sample_df.append(marks_df)

In [172]:
sample_df['BM25Okapi'] = None
sample_df['BM25L'] = None
sample_df['BM25Plus'] = None

In [173]:
for query in tqdm.tqdm_notebook(set(sample_df['query_id'])):
    docs_id = list(sample_df[sample_df['query_id'] == query]['url_id'])
    q = queries[queries['Id'] == query]['Query'][query]
    tokenized_query = q.split(" ")

    bm25_1 = bm25_Okapi.get_batch_scores(tokenized_query, docs_id)
    bm25_2 = bm25_L.get_batch_scores(tokenized_query, docs_id)
    bm25_3 = bm25_Plus.get_batch_scores(tokenized_query, docs_id)
    
    sample_df.loc[sample_df['query_id'] == query, 'BM25Okapi'] = bm25_1
    sample_df.loc[sample_df['query_id'] == query, 'BM25L'] = bm25_2
    sample_df.loc[sample_df['query_id'] == query, 'BM25Plus'] = bm25_3

/home/andrey/anaconda3/envs/matcher/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/6311 [00:00<?, ?it/s]

In [174]:
sample_df.to_csv('./BM25/BM25_Spelled_Stemmed.tsv',sep='\t',index=False)

### TF-IDF

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import sklearn.metrics.pairwise

In [12]:
# (Прогнать два раза - для speller и для stemmer)
# titles = pd.read_csv('./titles_speller_stemmer.tsv', sep='\t', header=None)
# titles.columns=['Id', 'Title']

# queries = pd.read_csv('./query_speller_stemmer.tsv', sep='\t', header=None)
# queries.columns = ['Id', 'Query']
# queries['Id'] = queries['Id'].apply(lambda x: int(x))

# titles['Title'] = titles['Title'].astype('str').apply(lambda x: x.strip())
# titles['Id'] = titles['Id'].apply(lambda x: int(x))
# titles = titles.sort_values(by=['Id'])

titles = pd.read_csv('./titles_speller.tsv', sep='\t', header=None)
titles.columns=['Id', 'Title']

queries = pd.read_csv('./query_speller.tsv', sep='\t', header=None)
queries.columns = ['Id', 'Query']
queries['Id'] = queries['Id'].apply(lambda x: int(x))

titles['Title'] = titles['Title'].astype('str').apply(lambda x: x.strip())
titles['Id'] = titles['Id'].apply(lambda x: int(x))
titles = titles.sort_values(by=['Id'])

In [13]:
sample_df = pd.read_csv('./Data/sample.csv', sep=',')
sample_df.columns = ['query_id', 'url_id']

marks_df = pd.read_csv('./Data/train.marks.tsv', sep='\t', header=None)
marks_df.columns = ['query_id', 'url_id', 'mark']
marks_df = marks_df.drop(columns=['mark'])

sample_df = sample_df.append(marks_df)

sample_df['TF_IDF'] = None
sample_df['Counter'] = None

In [14]:
corpus = titles['Title']

In [15]:
vectorizer_tf = TfidfVectorizer(ngram_range=(1, 4))
X_tfidf = vectorizer_tf.fit_transform(corpus)

vectorizer_count = CountVectorizer(ngram_range=(1, 4))
X_count = vectorizer_count.fit_transform(corpus)

print(X_count.shape)

(582167, 6046197)


In [16]:
for query in tqdm.tqdm_notebook(set(sample_df['query_id'])):
    docs_id = list(sample_df[sample_df['query_id'] == query]['url_id'])
    q = queries[queries['Id'] == query]['Query'][query]

    q_vec_idf = vectorizer_tf.transform([q])
    q_vec_count = vectorizer_count.transform([q])
    
    tf_idf = sklearn.metrics.pairwise.cosine_similarity(q_vec_idf, X_tfidf[docs_id])[0]
    count = sklearn.metrics.pairwise.cosine_similarity(q_vec_count, X_count[docs_id])[0]
    
    sample_df.loc[sample_df['query_id'] == query, 'TF_IDF'] = tf_idf
    sample_df.loc[sample_df['query_id'] == query, 'Counter'] = count

/home/andrey/anaconda3/envs/matcher/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/6311 [00:00<?, ?it/s]

In [17]:
sample_df.to_csv('./TFIDF/TF_IDF_Spelled.tsv',sep='\t',index=False)
#sample_df.to_csv('./TFIDF/TF_IDF_Spelled_Stemmed.tsv',sep='\t',index=False)